In [128]:
# 79-character line limit
######### ######### ######### ######### ######### ######### ######### #########
%reset -f

import numpy as np # The NumPy library
#import math # The math module, np includes it
from scipy.integrate import quad # Method for integration in scipy.integrate sub-package
from scipy.optimize import linprog


cells_num = 32

# The sizes are for UQs for number of cells: 4, 8, 16, 32, 64, 128, 256.

# The opitimized spaces for the Guassian distribution in a uniform quantizer(UQ).
all_size_gaus = np.array([0.9957, 0.5860, 0.3352, 0.1881, 0.1041, 0.0569, 0.0308])

# The opitimized spaces for the Laplacian distribution in a uniform quantizer(UQ).
all_size_laplacian = np.array(
                   [1.414, 1.0873, 0.8707, 0.7309, 0.6334, 0.5613, 0.5055])


'''The opitimized spaces for a Mixed Gaussian distribution in 
a uniform quantizer(UQ). This distribution is not symmetric and the values are
for for cells number 2, 4, 8, 16, 32, 64, 128, 256'''
all_boudary1s = [-0.05, -2.02, -3.17, -3.93, -4.49, -5.08, -5.16, -5.50]
all_deltas = [4, 1.81, 1, 0.54, 0.29, 0.16, 0.08, 0.05]

'''To change the symmetric pdf, two parts of code must be changed include the 
cell_size and pdf'''

cell_size = all_size_gaus[int(np.log2(cells_num))-2]
# cell_size = all_size_laplacian[int(np.log2(cells_num))-2]

# np.arange has rounding error issue.
# boundaries_symm = np.arange(-(cells_num/2-1) * cell_size, # For symmetric distributions
#                              (cells_num/2) * cell_size,
#                             cell_size)

# Boundaries for the symmetric distributions are computed.
boundaries_symm = np.linspace(-(cells_num/2-1) * cell_size, # For symmetric distributions
                               (cells_num/2-1) * cell_size,
                              cells_num-1)

# Boundaries for the asymmetric distributions are computed.
boundary1 = all_boudary1s[int(np.log2(cells_num))-1]
delta = all_deltas[int(np.log2(cells_num))-1]

boundaries_asymm = []
for i in range (0,cells_num-1):
        boundary = boundary1 + (i) * delta
        boundaries_asymm = np.append(boundaries_asymm, boundary)
        
'''To change the symmetric pdf to asymetric pdf, two parts of code 
must be changed include the boundaries and pdf'''        

# boundaries = boundaries_symm # For symmetric distributions
boundaries = boundaries_asymm # For asymetric distributions
        

n_inf = float("-inf")
p_inf = float("inf")

boundaries = np.insert(boundaries, 0, n_inf)
boundaries = np.append(boundaries, p_inf)

def pdf(x): # Defining the distribution
#     gaus_std = 1
#     gaus_mean = 0
#     pdf = 1/(gaus_std*np.sqrt(2*np.pi)) * \
#                   np.exp(-0.5*((x-gaus_mean)/gaus_std)**2) # Gaussian pdf
    
#     div = 1
#     pdf = 1/(2*div)*np.exp(-abs(x)/div) # Laplacian pdf

    Mean1 = -2
    Mean2 = 2
    stdev1 = 1
    stdev2 = 1
    alpha = 0.7
    pdf = (alpha)*(1/np.sqrt(2*np.pi*stdev1)*np.exp(-(x-Mean1)**2/(2*stdev1**2)))\
        + (1-alpha)*(1/np.sqrt(2*np.pi*stdev2)*np.exp(-(x-Mean2)**2/(2*stdev2**2)))
    
    return pdf

def xpdf(x):
    xpdf = x * pdf(x)
    return xpdf

def x2pdf (x):
    x2pdf = x * xpdf(x)
    return x2pdf


prbs = []
xprbs = []
x2prbs = []
cell_reps = []
for i in range (0, cells_num):
    cell_prb, integ_err = quad(pdf, boundaries[i], boundaries[i+1])
    prbs = np.append(prbs, cell_prb)
    
    cell_xprb, integ_err = quad(xpdf, boundaries[i], boundaries[i+1])
    xprbs = np.append(xprbs, cell_xprb)
    
    cell_x2prb, integ_err = quad(x2pdf, boundaries[i], boundaries[i+1])
    x2prbs = np.append(x2prbs, cell_x2prb)
    
    cell_rep = cell_xprb / cell_prb
    cell_reps = np.append(cell_reps, cell_rep)
    
    
'''The centroids and distortions of all possible pairs are calculated 
based on the strategy 1.'''

all_yijs = []
all_dijs = []
for i in range(0,cells_num-1):
    for j in range(i+1,cells_num):
        possible_yij = (xprbs[i] + xprbs[j])/(prbs[i] + prbs[j])
        all_yijs = np.append(all_yijs, possible_yij)
    
        possible_dij\
        = x2prbs[i] + possible_yij**2 * prbs[i] - possible_yij *2 * xprbs[i]\
        + x2prbs[j] + possible_yij**2 * prbs[j] - possible_yij *2 * xprbs[j]
        all_dijs = np.append(all_dijs, possible_dij)
        

# We implement the linear programming.
obj_coef = -all_dijs
all_length = int(cells_num * (cells_num-1) / 2)

A_eq = np.zeros([cells_num, all_length])
k = 0
for i in range(0, cells_num-1):
    for j in range(i+1,cells_num):
        A_eq[i,k]=1
        k+=1
k = 0
for j in range(1, cells_num):
    for i in range (j,cells_num):
        A_eq[i,k]=1
        k+=1

b_eq = np.ones([cells_num,])

result = linprog(obj_coef, A_eq=A_eq, b_eq=b_eq)
opt_xs = result.x
opt_pairs = np.round(result.x) # Pairs
opt_distortion = -result.fun # Value of the distortion

k = 0
xx = np.zeros([cells_num, cells_num])
for i in range(0,cells_num):
    for j in range(i+1,cells_num):
        xx[i,j] = opt_pairs[k]
        k +=1
xx = xx + np.transpose(xx)
yy = np.where(xx)
zz = np.append(yy[0],yy[1])
pairs = np.reshape(zz,[2,cells_num])
pairs = pairs[:,0:int(cells_num/2)]

print(opt_distortion)
# print(opt_xs)
# print(opt_pairs)
print('\n', pairs)

4.311099170444795

 [[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [30 31 29 28 21 22 23 24 25 26 27 20 19 18 17 16]]


In [129]:
all_yijkls = []
all_dijkls = []
for i in range (0, int(cells_num/2)-1):
    for j in range(i+1,int(cells_num/2)):
        possible_yijkl\
        = ((xprbs[pairs[0,i]] + xprbs[pairs[1,i]])
        + (xprbs[pairs[0,j]] + xprbs[pairs[1,j]]))\
        /((prbs[pairs[0,i]] + prbs[pairs[1,i]])
        +(prbs[pairs[0,j]] + prbs[pairs[1,j]])) 
        
        all_yijkls = np.append(all_yijkls, possible_yijkl)
        
        possible_dijkl\
        = x2prbs[pairs[0,i]] + possible_yijkl**2 * prbs[pairs[0,i]]\
        - possible_yijkl *2 * xprbs[pairs[0,i]]\
        + x2prbs[pairs[1,i]] + possible_yijkl**2 * prbs[pairs[1,i]]\
        - possible_yijkl *2 * xprbs[pairs[1,i]]\
        + x2prbs[pairs[0,j]] + possible_yijkl**2 * prbs[pairs[0,j]]\
        - possible_yijkl *2 * xprbs[pairs[0,j]]\
        + x2prbs[pairs[1,j]] + possible_yijkl**2 * prbs[pairs[1,j]]\
        - possible_yijkl *2 * xprbs[pairs[1,j]]
        
        all_dijkls = np.append(all_dijkls, possible_dijkl)
        
# We implement the linear programming.
quad_obj_coef = -all_dijkls
all_length_half = int(cells_num/2 * (cells_num/2-1) / 2)

A_eq_half = np.zeros([int(cells_num/2), all_length_half])

k = 0
for i in range(0, int(cells_num/2-1)):
    for j in range(i+1,int(cells_num/2)):
        A_eq_half[i,k] = 1
        k+=1
k = 0
for j in range(1, int(cells_num/2)):
    for i in range (j,int(cells_num/2)):
        A_eq_half[i,k] = 1
        k+=1

b_eq_half = np.ones([int(cells_num/2),])

quad_result = linprog(quad_obj_coef, A_eq=A_eq_half, b_eq=b_eq_half)
opt_xs_quad = quad_result.x
opt_quads = np.round(quad_result.x) # Quads
opt_distortion_quad = -quad_result.fun # Value of the distortion


print(opt_distortion_quad)
# print(opt_quads)

k = 0
xx = np.zeros([int(cells_num/2), int(cells_num/2)])
for i in range(0,int(cells_num/2)):
    for j in range(i+1,int(cells_num/2)):
        xx[i,j] = opt_quads[k]
        k +=1
        
# xx = xx + np.transpose(xx)
# yy = np.where(xx)
# zz = np.append(yy[0],yy[1])
# pairs_pairs = np.reshape(zz,[2,int(cells_num/2)])
# pairs_pairs = pairs_pairs[:,0:int(cells_num/4)]

pairs_pairs = np.array(np.where(xx == 1))
print('\n',pairs_pairs)

quads = np.zeros([4,int(cells_num/4)])
quads_row = []
for i in range(0,int(cells_num/4)):
    for j in range(0,2):
        quad = pairs[:,pairs_pairs[j,i]]
        quads_row = np.append(quads_row, quad)
        
quads = np.transpose(np.reshape(quads_row, [int(cells_num/4),4]))
print('\n',quads)

4.359325958861629

 [[ 0  1  2  5  6  7  8  9]
 [10  4  3 11 13 15 14 12]]

 [[ 0.  1.  2.  5.  6.  7.  8.  9.]
 [30. 31. 29. 22. 23. 24. 25. 26.]
 [10.  4.  3. 11. 13. 15. 14. 12.]
 [27. 21. 28. 20. 18. 16. 17. 19.]]


In [160]:
# print(all_dijkls[0]+all_dijkls[5])
# print(all_dijkls[1]+all_dijkls[4])
# print(all_dijkls[2]+all_dijkls[3])

In [117]:
pairs_pairs

array([[0, 1, 3, 4],
       [2, 5, 7, 6]])

In [131]:
# We calculate the distortion of 2MSBs encryption for NBIA.

nb_all_yijkls = []
nb_all_dijkls = []
power = np.log2(cells_num)
for i in range(0,int(cells_num/4)):
    j = int(i + 2**(power-2))
    k = int(i + 2**(power-2)*2)
    l = int(i + 2**(power-2)*3)
    print(i,j,k,l)
    
    nb_yijkl = (xprbs[i]+xprbs[j]+xprbs[k]+xprbs[l])\
             / (prbs[i]+prbs[j]+prbs[k]+prbs[l]) 
        
    nb_all_yijkls = np.append(nb_all_yijkls, nb_yijkl)
        
    nb_dijkl = x2prbs[i] + nb_yijkl**2 * prbs[i] - nb_yijkl *2 * xprbs[i]\
             + x2prbs[j] + nb_yijkl**2 * prbs[j] - nb_yijkl *2 * xprbs[j]\
             + x2prbs[k] + nb_yijkl**2 * prbs[k] - nb_yijkl *2 * xprbs[k]\
             + x2prbs[l] + nb_yijkl**2 * prbs[l] - nb_yijkl *2 * xprbs[l]
    
    nb_all_dijkls = np.append(nb_all_dijkls, nb_dijkl)
    
nb_distortion = np.sum(nb_all_dijkls)
print('\n',nb_distortion)

0 8 16 24
1 9 17 25
2 10 18 26
3 11 19 27
4 12 20 28
5 13 21 29
6 14 22 30
7 15 23 31

 4.355823371658127
